# Loading most recent data from postgres table

## Installing needed packages and connections

In [1]:
# Install libraries
!pip install psycopg2-binary
!pip install requests
!pip install sqlalchemy --upgrade
!pip install dask --upgrade

## Load data and analytics using Dask

In [29]:
import os
import dask.dataframe as dd
from api_query import api_query_call

df_pandas = api_query_call("SELECT * FROM uniswap_table")


In [61]:
df = dd.from_pandas(df_pandas, npartitions=10)
df.head()

,tradeindex,block_timestamp_time,block_height,exchange_fullname,protocol,timeinterval_minute,baseamount,quoteamount,trades,quoteprice,...,open_price,close_price,basecurrency_symbol,basecurrency_address,quotecurrency_symbol,quotecurrency_address,transaction_hash,transaction_gas,transaction_to_address,transaction_txfrom_address
0,8,2023-05-08 00:26:35,17212211,Uniswap,Uniswap,2023-05-08 00:26:00,191.130888692456,190.9673278680587,1,0.0005308734179599499,...,0.0005308734179599499,0.0005308734179599499,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,ETH,-,0x19535811991530dac2e9c71c20716c3d84c7ef2751c3...,347106,0xf015ff012cdad388a857d9163cb01e8dc29f310c,0xc723f2c210c4d29cfe35209340a6fb766d956982
1,8,2023-05-08 00:26:35,17212211,Uniswap,Uniswap,2023-05-08 00:26:00,190.9673278680587,191.130888692456,1,1883.6882129883586,...,1883.6882129883586,1883.6882129883586,ETH,-,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x19535811991530dac2e9c71c20716c3d84c7ef2751c3...,347106,0xf015ff012cdad388a857d9163cb01e8dc29f310c,0xc723f2c210c4d29cfe35209340a6fb766d956982
2,4,2023-05-08 00:02:23,17212091,Uniswap,Uniswap,2023-05-08 00:02:00,2763.2171636340645,2747.5099529385575,1,0.0005283747783757336,...,0.0005283747783757336,0.0005283747783757336,DAI,0x6b175474e89094c44da98b954eedeac495271d0f,ETH,-,0xcd95055fbed6f391ae5f5a80c5ab9bead46ab5ddd18f...,308002,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0xd7e1236c08731c3632519dcd1a581bfe6876a3b2
3,4,2023-05-08 00:02:23,17212091,Uniswap,Uniswap,2023-05-08 00:02:00,2747.5099529385575,2763.2171636340645,1,1892.5960150370538,...,1892.5960150370538,1892.5960150370538,ETH,-,DAI,0x6b175474e89094c44da98b954eedeac495271d0f,0xcd95055fbed6f391ae5f5a80c5ab9bead46ab5ddd18f...,308002,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0xd7e1236c08731c3632519dcd1a581bfe6876a3b2
4,4,2023-05-08 00:01:11,17212085,Uniswap,Uniswap,2023-05-08 00:01:00,3499.3400938542027,3481.433844317239,1,0.0005286092784797666,...,0.0005286092784797666,0.0005286092784797666,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,ETH,-,0x25a892f938c73a8f54dc1a6ece913fe1c4cf280dab5f...,142623,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0xd7e1236c08731c3632519dcd1a581bfe6876a3b2


In [33]:
#Statistics
sum_stats = df_dask.describe().compute()
print(sum_stats)

       tradeindex  block_height  trades  transaction_gas
count  950.000000  9.500000e+02   950.0     9.500000e+02
mean     6.040000  1.714295e+07     1.0     2.662814e+05
std      4.893005  3.572817e+04     0.0     2.100202e+05
min      2.000000  1.708408e+07     1.0     5.184700e+04
25%      4.000000  1.711956e+07     1.0     1.685050e+05
50%      5.000000  1.713821e+07     1.0     2.425660e+05
75%     10.000000  1.717466e+07     1.0     3.868300e+05
max     41.000000  1.721295e+07     1.0     2.036563e+06


In [76]:
print('Most common "from" wallet addresses:')
df['transaction_txfrom_address'].value_counts().head()

Most common "from" wallet addresses:


0xd7e1236c08731c3632519dcd1a581bfe6876a3b2    148
0xb543f8d66765c7074d1d77147d7e07880c96908f     56
0x2ee36e41387f87b7e6f678a86d1e575b23b996f5     34
0xa06c3c08a19e51b33309eddfb356c33ead8517a3     34
0xbda27a23fb386024c495ad2f6b834cbfe829b913     18
Name: transaction_txfrom_address, dtype: int64

In [74]:
print('Check that no overlaps of same wallet in to and from category:')
sum(df['transaction_txfrom_address'] == df['transaction_to_address'])

Check that no overlaps of same wallet in to and from category:


0

In [59]:
# top transactions and addresses
counts_address = df['transaction_to_address'].value_counts()

print(counts_address.compute()[:10])

0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0    164
0xdef1c0ded9bec7f1a1670819833240f027b25eff    128
0x0eae044f00b0af300500f090ea00027097d03000     86
0x0000000000007f150bd6f54c40a34d7c3d5e9f56     76
0x0000000000a84d1a9b0063a910315c7ffa9cd248     76
0xe66b31678d6c16e9ebf358268a790b763c133750     70
0x1111111254eeb25477b68fb85ed929f73a960582     46
0x76f4eed9fe41262669d0250b2a97db79712ad855     46
0xe4000004000bd8006e00720000d27d1fa000d43e     42
0x881d40237659c251811cec9c364ef91dc08d300c     36
Name: transaction_to_address, dtype: int64


In [75]:
df_top1 = df[df.transaction_to_address == counts_address.index.compute()[0]]   
df_top1.describe().compute()

,tradeindex,block_height,trades,transaction_gas
count,164.000000,1.640000e+02,164.0,164.000000
mean,6.853659,1.714627e+07,1.0,219741.597561
std,4.461483,3.797255e+04,0.0,121427.369346
min,3.000000,1.708408e+07,1.0,117810.000000
25%,4.250000,1.712058e+07,1.0,141635.000000
50%,6.000000,1.713933e+07,1.0,177759.000000
75%,8.000000,1.717987e+07,1.0,308002.000000
max,25.000000,1.721209e+07,1.0,730533.000000
